In [1]:
# battery_swap_optimizer.py

import pandas as pd
import numpy as np
from geopy.distance import geodesic
from datetime import datetime, timedelta
import random

# --- CONFIGURATION ---
NUM_RIDERS = 100
NUM_STATIONS = 3
PUNE_CENTER = (18.5204, 73.8567)  # Central Pune coordinates
SOC_USAGE_PER_KM = 4  # 4% SOC per km
MAX_QUEUE_LEN = 5
SWAP_DURATION_MIN = 4

# --- MOCK DATA GENERATION ---
def generate_riders(num=NUM_RIDERS):
    riders = []
    for i in range(num):
        rider_id = f"R{i:03}"
        lat = PUNE_CENTER[0] + np.random.uniform(-0.02, 0.02)
        lng = PUNE_CENTER[1] + np.random.uniform(-0.02, 0.02)
        soc_pct = np.random.randint(5, 100)
        status = random.choice(["idle", "on_gig"])
        km_to_finish = round(np.random.uniform(0.5, 5.0), 2) if status == "on_gig" else 0.0
        est_finish_ts = datetime.now() + timedelta(minutes=random.randint(5, 30))
        riders.append({
            "rider_id": rider_id,
            "lat": lat,
            "lng": lng,
            "soc_pct": soc_pct,
            "status": status,
            "km_to_finish": km_to_finish,
            "est_finish_ts": est_finish_ts,
        })
    return pd.DataFrame(riders)

def generate_stations(num=NUM_STATIONS):
    stations = []
    for i in range(num):
        station_id = f"S_{chr(65+i)}"
        lat = PUNE_CENTER[0] + np.random.uniform(-0.03, 0.03)
        lng = PUNE_CENTER[1] + np.random.uniform(-0.03, 0.03)
        queue_len = random.randint(0, MAX_QUEUE_LEN - 1)
        stations.append({
            "station_id": station_id,
            "lat": lat,
            "lng": lng,
            "queue_len": queue_len,
        })
    return pd.DataFrame(stations)

# --- OPTIMIZATION LOGIC ---
def assign_swap_station(riders_df, stations_df):
    output = []
    for _, rider in riders_df.iterrows():
        if rider['soc_pct'] < 10 or \
           (rider['status'] == 'on_gig' and rider['soc_pct'] - rider['km_to_finish'] * SOC_USAGE_PER_KM < 10):

            # Calculate distance to each station
            stations_df['distance_km'] = stations_df.apply(
                lambda s: geodesic((rider['lat'], rider['lng']), (s['lat'], s['lng'])).km, axis=1
            )
            # Filter by queue capacity
            eligible_stations = stations_df[stations_df['queue_len'] < MAX_QUEUE_LEN]
            if eligible_stations.empty:
                continue

            # Choose nearest station
            nearest_station = eligible_stations.sort_values(by='distance_km').iloc[0]

            # Estimate timestamps
            travel_time_min = nearest_station['distance_km'] / 0.3 * 60  # 18 km/h avg
            depart_ts = datetime.now()
            arrive_ts = depart_ts + timedelta(minutes=travel_time_min)
            swap_start_ts = arrive_ts
            swap_end_ts = swap_start_ts + timedelta(minutes=SWAP_DURATION_MIN)

            eta_back_lat = rider['lat']
            eta_back_lng = rider['lng']

            output.append({
                "rider_id": rider['rider_id'],
                "station_id": nearest_station['station_id'],
                "depart_ts": depart_ts,
                "arrive_ts": arrive_ts,
                "swap_start_ts": swap_start_ts,
                "swap_end_ts": swap_end_ts,
                "eta_back_lat": eta_back_lat,
                "eta_back_lng": eta_back_lng
            })

            # Update queue length
            stations_df.loc[stations_df['station_id'] == nearest_station['station_id'], 'queue_len'] += 1

    return pd.DataFrame(output)

In [2]:
if __name__ == "__main__":
    riders = generate_riders()
    stations = generate_stations()
    plan = assign_swap_station(riders, stations)

    # Save outputs
    riders.to_csv("mock_riders.csv", index=False)
    stations.to_csv("mock_stations.csv", index=False)
    plan.to_csv("plan_output.csv", index=False)

    print("Optimization complete. Output saved to plan_output.csv")


Optimization complete. Output saved to plan_output.csv


In [ ]:
mock_rider = pd.read_csv("mock_riders.csv")
mock_rider.head()

,rider_id,lat,lng,soc_pct,status,km_to_finish,est_finish_ts
0,R000,18.530225,73.863953,8,idle,0.00,2025-06-20 08:26:35.696719
1,R001,18.534240,73.859817,40,on_gig,4.07,2025-06-20 08:17:35.696809
2,R002,18.521940,73.844854,97,on_gig,2.18,2025-06-20 08:10:35.696852
3,R003,18.506633,73.858067,37,idle,0.00,2025-06-20 08:12:35.696877
4,R004,18.509038,73.842454,10,on_gig,1.86,2025-06-20 08:07:35.696911


In [ ]:
mock_stations = pd.read_csv("mock_stations.csv")
mock_stations.head()

,station_id,lat,lng,queue_len,distance_km
0,S_A,18.541028,73.873568,5,3.924207
1,S_B,18.546333,73.849989,5,3.975447
2,S_C,18.529277,73.865281,5,2.359913


In [ ]:
plan_output = pd.read_csv("plan_output.csv")
plan_output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   rider_id       14 non-null     object 
 1   station_id     14 non-null     object 
 2   depart_ts      14 non-null     object 
 3   arrive_ts      14 non-null     object 
 4   swap_start_ts  14 non-null     object 
 5   swap_end_ts    14 non-null     object 
 6   eta_back_lat   14 non-null     float64
 7   eta_back_lng   14 non-null     float64
dtypes: float64(2), object(6)
memory usage: 1.0+ KB


In [3]:
# gradio_ui.py

import gradio as gr
#from battery_swap_optimizer import generate_riders, generate_stations, assign_swap_station
import pandas as pd

def run_optimizer(num_riders, num_stations):
    riders_df = generate_riders(num_riders)
    stations_df = generate_stations(num_stations)
    plan_df = assign_swap_station(riders_df, stations_df)

    # Save results
    riders_csv = riders_df.to_csv(index=False)
    stations_csv = stations_df.to_csv(index=False)
    plan_csv = plan_df.to_csv(index=False)

    return riders_df, stations_df, plan_df, plan_csv

with gr.Blocks(title="Battery Swap Routing Optimizer") as demo:
    gr.Markdown("# 🔋 Battery-Swap Routing Optimizer\nMock rider + station data, optimized swap plan")

    with gr.Row():
        num_riders = gr.Slider(minimum=10, maximum=200, value=100, label="Number of Riders")
        num_stations = gr.Slider(minimum=3, maximum=10, value=6, label="Number of Swap Stations")

    run_btn = gr.Button("Run Optimizer")

    with gr.Tab("🧍 Riders Data"):
        rider_output = gr.Dataframe(interactive=False)
    with gr.Tab("🏪 Stations Data"):
        station_output = gr.Dataframe(interactive=False)
    with gr.Tab("✅ Swap Plan Output"):
        plan_output = gr.Dataframe(interactive=False)
        csv_output = gr.File(label="Download Plan CSV")

    def run_all(riders, stations):
        riders_df, stations_df, plan_df, plan_csv = run_optimizer(riders, stations)
        with open("plan_output.csv", "w") as f:
            f.write(plan_csv)
        return riders_df, stations_df, plan_df, "plan_output.csv"

    run_btn.click(run_all, inputs=[num_riders, num_stations], outputs=[rider_output, station_output, plan_output, csv_output])

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://012995f5015a09c91b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
